# Swaps

In [ ]:
#| default_exp swap

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import copy
from dataclasses import dataclass
from sugar.config import XCHAIN_GAS_LIMIT_UPPERBOUND
from sugar.quote import Quote, SuperswapQuote, pack_path
from sugar.helpers import apply_slippage, ICACallData, hash_ICA_calls, to_bytes32, to_bytes32_str, MAX_UINT256
from sugar.helpers import ADDRESS_ZERO
from sugar.token import Token
from sugar.pool import LiquidityPoolForSwap
from enum import IntEnum
from typing import List, Any, Tuple, Optional
from eth_abi import encode
from eth_abi.packed import encode_packed
from web3 import Web3
from sugar.abi import get_abi, erc20_abi


In [ ]:
#| export

class CommandType(IntEnum):
    V3_SWAP_EXACT_IN = 0x00
    V3_SWAP_EXACT_OUT = 0x01
    SWEEP = 0x04
    V2_SWAP_EXACT_IN = 0x08
    V2_SWAP_EXACT_OUT = 0x09
    WRAP_ETH = 0x0b
    UNWRAP_WETH = 0x0c
    TRANSFER_FROM = 0x07
    BRIDGE_TOKEN = 0x12
    EXECUTE_CROSS_CHAIN = 0x13
    EXECUTE_SUB_PLAN = 0x21


class BridgeType(IntEnum):
    """
    Enum for different bridge types.
    """
    HYP_XERC20 = 0x01
    XVELO = 0x02

FLAG_ALLOW_REVERT = 0x80

# Define ABI types for each command
ABI_DEFINITION = {
    CommandType.V3_SWAP_EXACT_IN: [
        "address", # recipient
        "uint256", # amountIn
        "uint256", # amountOutMin
        "bytes", # path
        "bool", # payerIsUser
        "bool"  # isUni
    ],
    CommandType.V2_SWAP_EXACT_IN: [
        "address",
        "uint256",
        "uint256",
        "bytes",
        "bool",
        "bool"  # isUni
    ],
    CommandType.V2_SWAP_EXACT_OUT: [
        "address",
        "uint256",
        "uint256",
        "bytes",
        "bool",
        "bool"  # isUni
    ],
    CommandType.V3_SWAP_EXACT_OUT: [
        "address",
        "uint256",
        "uint256",
        "bytes",
        "bool",
        "bool"  # isUni
    ],
    CommandType.WRAP_ETH: [
        "address",
        "uint256"
    ],
    CommandType.UNWRAP_WETH: [
        "address",
        "uint256"
    ],
    CommandType.SWEEP: [
        "address",
        "address",
        "uint256"
    ],
    CommandType.TRANSFER_FROM: [
        "address",  # token
        "address",  # recipient
        "uint256"   # amount
    ],
    CommandType.BRIDGE_TOKEN: [
        "uint8",    # bridgeType
        "address",  # recipient
        "address",  # token
        "address",  # bridge
        "uint256",  # amount
        "uint256",  # msgFee
        "uint32",   # domain
        "bool"      # payerIsUser
    ],
    CommandType.EXECUTE_SUB_PLAN: [
        "bytes",    
        "bytes[]"
    ],
    CommandType.EXECUTE_CROSS_CHAIN: [
        "uint32",   # domain
        "address",  # icaRouter
        "bytes32",  # remoteRouter
        "bytes32",  # ism
        "bytes32",  # commitment
        "uint256",  # msgFee
        "address",  # hook
        "bytes"
    ]
}

class RoutePlanner:
    def __init__(self):
        """Initialize a new RoutePlanner"""
        self.commands = "0x"
        self.inputs: List[bytes] = []
        self.bytes_commands: List[bytes] = []

    def add_command(self, command_type: CommandType, parameters: List[Any], allow_revert = False) -> None:
        """
        Add a command to the route planner
        
        Args:
            command_type: Type of command to add
            parameters: Parameters for the command
        """
        # Get the ABI definition for this command
        abi_types = ABI_DEFINITION[command_type]
        self.inputs.append(encode(abi_types, parameters))
        # Add command byte to commands
        command_type = command_type | FLAG_ALLOW_REVERT if allow_revert else command_type
        command_hex = format(command_type, '02x')
        self.commands = self.commands + command_hex
        # TODO: figure out why we keep this alongside command_hex
        self.bytes_commands.append(command_hex)

    def get_encoded_commands(self) -> str: return self.commands
    
    def get_encoded_inputs(self) -> List[bytes]: return self.inputs

    # using this for testing
    def get_pretty_encoded_inputs(self) -> List[str]: return list(map(lambda i: "0x" + i.hex(), self.get_encoded_inputs())) 

In [ ]:

def get_command_byte(command: int):
  return format(command, '02x')

get_command_byte(CommandType.V2_SWAP_EXACT_IN | FLAG_ALLOW_REVERT)

'88'

In [ ]:
#| export

# Constants
CONTRACT_BALANCE = int("0x8000000000000000000000000000000000000000000000000000000000000000", 16)

def setup_planner(quote: Quote, slippage: float, account: str, router_address: str) -> RoutePlanner:
    """Setup route planner with the given quote and chain"""

    route_planner = RoutePlanner()
    min_amount_out = apply_slippage(quote.amount_out, slippage)

    tokens_come_from_contract = quote.input.amount_in == CONTRACT_BALANCE
    
    # Handle wrapped native token if needed
    if quote.from_token.wrapped_token_address:
        # When trading from native token, wrap token first
        route_planner.add_command(CommandType.WRAP_ETH, [router_address, quote.amount_in])
        tokens_come_from_contract = True
    
    # Group nodes by pool type (v2 or v3)
    grouped_nodes: List[List[Tuple[LiquidityPoolForSwap, bool]]] = []

    for node in quote.path:
        if not grouped_nodes: grouped_nodes.append([node])
        elif node[0].type < 1:
            # Current node is a v2 pool
            if float(grouped_nodes[-1][0][0].type) < 1: grouped_nodes[-1].append(node)
            else: grouped_nodes.append([node])
        else:
            # Current node is a v3 pool
            if grouped_nodes[-1][0][0].type >= 1: grouped_nodes[-1].append(node)
            else: grouped_nodes.append([node])
    
    if len(grouped_nodes) == 1:
        # All nodes belong to the same pool type
        nodes = grouped_nodes[0]
        is_v2_pool = float(nodes[0][0].type) < 1
        
        route_planner.add_command(
            CommandType.V2_SWAP_EXACT_IN if is_v2_pool else CommandType.V3_SWAP_EXACT_IN,
            [
                # Where should money go?
                router_address if quote.to_token.wrapped_token_address else account,
                quote.amount_in,
                min_amount_out,
                pack_path(nodes, for_swap=True).encoded,
                not tokens_come_from_contract,
                False, # isUni
            ]
        )
    else:
        # Mixed v2 and v3 pools
        first_batch = grouped_nodes[0]
        last_batch = grouped_nodes[-1]
        rest = grouped_nodes[1:-1]
        
        # Handle first batch
        is_first_batch_v2 = not first_batch[0][0].is_cl
        next_batch = rest[0] if rest else last_batch
        
        route_planner.add_command(
            CommandType.V2_SWAP_EXACT_IN if is_first_batch_v2 else CommandType.V3_SWAP_EXACT_IN,
            [
                router_address if is_first_batch_v2 else next_batch[0][0].lp,
                quote.amount_in,
                0,  # No expectations on min amount out for first batch
                pack_path(first_batch, for_swap=True).encoded,
                not tokens_come_from_contract,
                False,  # isUni
            ]
        )
        
        # Handle middle batches
        for idx, batch in enumerate(rest):
            is_batch_v2 = not batch[0][0].is_cl
            next_batch = rest[idx + 1] if idx + 1 < len(rest) else last_batch
            
            route_planner.add_command(
                CommandType.V2_SWAP_EXACT_IN if is_batch_v2 else CommandType.V3_SWAP_EXACT_IN,
                [
                    router_address if is_batch_v2 else next_batch[0][0].lp,
                    0 if is_batch_v2 else CONTRACT_BALANCE,
                    0,  # No expectations for middle batches
                    pack_path(batch, for_swap=True).encoded,
                    False,  # Money comes from contract
                    False,  # isUni
                ]
            )
        
        # # Handle last batch
        is_last_batch_v2 = not last_batch[0][0].is_cl
        
        route_planner.add_command(
            CommandType.V2_SWAP_EXACT_IN if is_last_batch_v2 else CommandType.V3_SWAP_EXACT_IN,
            [
                router_address if quote.to_token.wrapped_token_address else account,
                0 if is_last_batch_v2 else CONTRACT_BALANCE,
                min_amount_out,
                pack_path(last_batch, for_swap=True).encoded,
                False,  # Money comes from contract
                False,  # isUni
            ]
        )
    
    # Handle unwrapping WETH if needed
    if quote.to_token.wrapped_token_address: route_planner.add_command(CommandType.UNWRAP_WETH, [account, min_amount_out])
    
    return route_planner

## Superswaps 🐝

In [ ]:
#| export

@dataclass(frozen=True)
class SuperSwapDataInput:
    from_token: Token
    to_token: Token
    from_bridge_token: Token
    to_bridge_token: Token
    account: str
    user_ICA: str
    user_ICA_balance: int
    origin_domain: int
    origin_bridge: str
    origin_hook: str
    origin_ICA_router: str
    destination_ICA_router: str
    destination_router: str
    destination_domain: int
    slippage: float
    swapper_contract_addr: str
    salt: str
    bridge_fee: int 
    xchain_fee: int
    destination_quote: Optional[Quote]

    @staticmethod
    def build(
            quote: SuperswapQuote, account: str,
            user_ICA: str, user_ICA_balance: int,
            origin_domain: int, origin_bridge: str, origin_hook: str, origin_ICA_router: str,
            destination_ICA_router: str, destination_router: str, destination_domain: int,
            slippage: float,
            swapper_contract_addr: str, salt: str,
            bridge_fee: int, xchain_fee: int
        ) -> 'SuperSwapDataInput':
        return SuperSwapDataInput(
            from_token=quote.from_token, to_token=quote.to_token, from_bridge_token=quote.from_bridge_token, to_bridge_token=quote.to_bridge_token,
            account=account, user_ICA=user_ICA, user_ICA_balance=user_ICA_balance,
            origin_domain=origin_domain, origin_bridge=origin_bridge, origin_hook=origin_hook, origin_ICA_router=origin_ICA_router,
            destination_ICA_router=destination_ICA_router, destination_router=destination_router, destination_domain=destination_domain,
            slippage=slippage,
            swapper_contract_addr=swapper_contract_addr, salt=salt,
            bridge_fee=bridge_fee, xchain_fee=xchain_fee,
            destination_quote=quote.destination_quote
        )

@dataclass(frozen=True)
class SuperSwapData:
    destination_planner: RoutePlanner
    calls: List[ICACallData]
    origin_domain: int
    salt: str
    needs_relay: bool
    commitment_hash: Optional[bytes] 

Superswap data helper to from super quote to superswap, baby

In [ ]:
#| export

def build_super_swap_data(i: SuperSwapDataInput) -> SuperSwapData:
    d_quote, account, slippage, swap_contract_addr = i.destination_quote, i.account, i.slippage, i.swapper_contract_addr

    # TODO: figure out if destination quote should come with tweaked amount
    if d_quote: 
        d_quote_with_max_amount_in = copy.deepcopy(d_quote)
        d_quote_with_max_amount_in.input.amount_in = CONTRACT_BALANCE
    
    destination_chain_swap_plan = setup_planner(d_quote_with_max_amount_in, slippage, account, swap_contract_addr) if d_quote else None

    swap_subplan_cmds = None

    if destination_chain_swap_plan:
        swap_subplan_cmds = destination_chain_swap_plan.get_encoded_commands()
        swap_subplan_cmds = "0x" + format(CommandType.TRANSFER_FROM, '02x') + swap_subplan_cmds.replace('0x', '')
    
    swap_subplan_cmds = bytes.fromhex(swap_subplan_cmds.replace('0x', '')) if swap_subplan_cmds else None

    destination_transfer_args = encode(ABI_DEFINITION[CommandType.TRANSFER_FROM], [
        i.to_bridge_token.token_address, i.destination_router, CONTRACT_BALANCE
    ])
    # Encode fallback transfer
    fallback_transfer_cmd = encode_packed(["bytes1"], [bytes([CommandType.TRANSFER_FROM])])
    fallback_transfer_args = encode(
        ABI_DEFINITION[CommandType.TRANSFER_FROM],
        [i.to_bridge_token.token_address, account, CONTRACT_BALANCE]
    )
    destination_inputs = [
        encode(ABI_DEFINITION[CommandType.EXECUTE_SUB_PLAN], [swap_subplan_cmds, [destination_transfer_args] + destination_chain_swap_plan.inputs]),
        encode(ABI_DEFINITION[CommandType.EXECUTE_SUB_PLAN], [fallback_transfer_cmd, [fallback_transfer_args]]),
    ] if swap_subplan_cmds else None

    calls = []

    if destination_inputs:
        # Encode Sub Plan commands
        destination_cmds = encode_packed(
            ["bytes", "bytes"],
            [
                bytes([CommandType.EXECUTE_SUB_PLAN | FLAG_ALLOW_REVERT]), 
                bytes([CommandType.EXECUTE_SUB_PLAN | FLAG_ALLOW_REVERT])
            ]
        )
        # ICA approves router to withdraw a very high amount
        calls.append(ICACallData(
            to=to_bytes32_str(i.to_bridge_token.token_address),
            value=0,
            data=Web3().eth.contract(abi=erc20_abi).encode_abi("approve", args=[i.destination_router, MAX_UINT256])
        ))
        # destination chain swap
        calls.append(ICACallData(
            to=to_bytes32_str(i.destination_router),
            value=0,
            data=Web3().eth.contract(abi=get_abi("swapper")).encode_abi("execute", args=[destination_cmds, destination_inputs])
        ))
        # reset approval back to zero
        calls.append(ICACallData(
            to=to_bytes32_str(i.to_bridge_token.token_address),
            value=0,
            data=Web3().eth.contract(abi=erc20_abi).encode_abi("approve", args=[i.destination_router, 0])
        ))

    commitment_hash = hash_ICA_calls(calls, i.salt)
    needs_relay = i.to_token.token_address != i.to_bridge_token.token_address

    destination_chain_planner = RoutePlanner()

    # TODO: confirm this is OK (destination quote check)
    # i think this works because bridge -> bridge is handled in its own way
    starts_with_bridge_token = (i.from_token.token_address == i.from_bridge_token.token_address) and d_quote is not None

    # bridge command
    destination_chain_planner.add_command(CommandType.BRIDGE_TOKEN, [
        BridgeType.HYP_XERC20,
        i.user_ICA if needs_relay else i.account,
        i.from_bridge_token.token_address,
        i.origin_bridge,
        d_quote.amount_in if starts_with_bridge_token else CONTRACT_BALANCE,
        i.bridge_fee,
        i.destination_domain,
        starts_with_bridge_token,
    ])

    if needs_relay:
        variant = 1
        hook_metadata = encode_packed(["uint16", "uint256", "uint256", "address"], [
            variant, i.xchain_fee, XCHAIN_GAS_LIMIT_UPPERBOUND, i.account 
        ])
        # destination swap command
        destination_chain_planner.add_command(CommandType.EXECUTE_CROSS_CHAIN, [
            i.destination_domain,
            i.origin_ICA_router,
            to_bytes32(i.destination_ICA_router),
            to_bytes32(ADDRESS_ZERO),
            commitment_hash,
            i.xchain_fee,
            i.origin_hook,
            hook_metadata
        ])

    return SuperSwapData(
        destination_planner=destination_chain_planner,
        calls=calls,
        origin_domain=i.origin_domain,
        salt=i.salt,
        needs_relay=needs_relay,
        commitment_hash=commitment_hash if needs_relay else None
    )

In [ ]:
from fastcore.test import test_eq
from sugar.token import Token
from sugar.quote import QuoteInput
from sugar.helpers import parse_ether

## Test planner

Let's test a few combinations of quote paths

In [ ]:
account = '0x533cf9fb379488ffe0b1065c42c744fbd4b0e1a3'
router = '0x4bF3E32de155359D1D75e8B474b66848221142fc'
eth = Token(chain_id="10", chain_name="OP", token_address='ETH', symbol='ETH', decimals=18, listed=True, wrapped_token_address='0x4200000000000000000000000000000000000006')
velo = Token(chain_id="10", chain_name="OP", token_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', symbol='VELO', decimals=18, listed=True, wrapped_token_address=None)
usdc = Token(chain_id="10", chain_name="OP", token_address='0x7F5c764cBc14f9669B88837ca1490cCa17c31607', symbol='USDC', decimals=6, listed=True, wrapped_token_address=None)

def setup_quote(path: List[Tuple[LiquidityPoolForSwap, bool]], from_token: Token, to_token: Token, amount_in: int, amount_out: int) -> Quote:
    return Quote(input=QuoteInput(from_token=from_token, to_token=to_token, path=path, amount_in=amount_in), amount_out=amount_out)

# simple quote with 1 v2 pool
unstable_v2 = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address), False)
    ],
    from_token=velo,
    to_token=usdc,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=unstable_v2, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x08')
test_eq(inputs, [
    "0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a30000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000299560e827af36c94d2ac33a39bce1fe78631088db007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000"
])

# v2 pool with native token wrap

unstable_v2_with_token_wrap = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=eth.wrapped_token_address, token1_address=usdc.token_address), False)
    ],
    from_token=eth,
    to_token=usdc,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=unstable_v2_with_token_wrap, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x0b08')
test_eq(inputs, ['0x0000000000000000000000004bf3e32de155359d1d75e8b474b66848221142fc0000000000000000000000000000000000000000000000000000000000000005', '0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a30000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000294200000000000000000000000000000000000006007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000'])

# v2 with native token unwrap

unstable_v2_with_token_unwrap = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=eth.wrapped_token_address), False)
    ],
    from_token=velo,
    to_token=eth,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=unstable_v2_with_token_unwrap, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x080c')
test_eq(inputs, ['0x0000000000000000000000004bf3e32de155359d1d75e8b474b66848221142fc0000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000299560e827af36c94d2ac33a39bce1fe78631088db0042000000000000000000000000000000000000060000000000000000000000000000000000000000000000', '0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a3000000000000000000000000000000000000000000000000000000000000000a'])

# simple v3 pool swap
simple_v3 = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=100, token0_address=velo.token_address, token1_address=usdc.token_address), False)
    ],
    from_token=velo,
    to_token=usdc,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=simple_v3, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x00')
test_eq(inputs, [
    "0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a30000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002b9560e827af36c94d2ac33a39bce1fe78631088db0000647f5c764cbc14f9669b88837ca1490cca17c31607000000000000000000000000000000000000000000"
])

# hybrid v2 + v2 + v3
v2_v2_v3 = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=100, token0_address=velo.token_address, token1_address=usdc.token_address), False)
    ],
    from_token=velo,
    to_token=usdc,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=v2_v2_v3, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x0800')
test_eq(inputs, ['0x0000000000000000000000004bf3e32de155359d1d75e8b474b66848221142fc0000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003e9560e827af36c94d2ac33a39bce1fe78631088db007f5c764cbc14f9669b88837ca1490cca17c31607007f5c764cbc14f9669b88837ca1490cca17c316070000', '0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a38000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002b9560e827af36c94d2ac33a39bce1fe78631088db0000647f5c764cbc14f9669b88837ca1490cca17c31607000000000000000000000000000000000000000000'])

# hybrid v2 + v3 + v2
v2_v3_v2 = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592a11', type=100, token0_address=velo.token_address, token1_address=usdc.token_address), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address), False)
    ],
    from_token=velo,
    to_token=usdc,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=v2_v3_v2, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x080008')
test_eq(inputs, ['0x0000000000000000000000004bf3e32de155359d1d75e8b474b66848221142fc0000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000299560e827af36c94d2ac33a39bce1fe78631088db007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000', '0x000000000000000000000000ec3d9098bd40ec741676fc04d4bd26bccf592aa38000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002b9560e827af36c94d2ac33a39bce1fe78631088db0000647f5c764cbc14f9669b88837ca1490cca17c31607000000000000000000000000000000000000000000', '0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a30000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000299560e827af36c94d2ac33a39bce1fe78631088db007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000'])

# super convoluted v3 + v2 + v3 + v3 + v2
v3_v2_v3_v3_v2 = setup_quote(
    path=[
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592a11', type=100, token0_address=velo.token_address, token1_address=usdc.token_address), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592a11', type=100, token0_address=velo.token_address, token1_address=usdc.token_address), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592a11', type=100, token0_address=velo.token_address, token1_address=usdc.token_address), False),
        (LiquidityPoolForSwap(chain_id="10", chain_name="OP", lp='0xec3d9098BD40ec741676fc04D4bd26BCCF592aa3', type=-1, token0_address=velo.token_address, token1_address=usdc.token_address), False)
    ],
    from_token=velo,
    to_token=usdc,
    amount_in=5,
    amount_out=10
)

planner = setup_planner(quote=v3_v2_v3_v3_v2, slippage=0.01, account=account, router_address=router)
commands, inputs = planner.get_encoded_commands(), planner.get_pretty_encoded_inputs()

test_eq(commands, '0x00080008')
test_eq(inputs, ['0x000000000000000000000000ec3d9098bd40ec741676fc04d4bd26bccf592aa30000000000000000000000000000000000000000000000000000000000000005000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002b9560e827af36c94d2ac33a39bce1fe78631088db0000647f5c764cbc14f9669b88837ca1490cca17c31607000000000000000000000000000000000000000000', '0x0000000000000000000000004bf3e32de155359d1d75e8b474b66848221142fc0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000299560e827af36c94d2ac33a39bce1fe78631088db007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000', '0x000000000000000000000000ec3d9098bd40ec741676fc04d4bd26bccf592aa38000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000429560e827af36c94d2ac33a39bce1fe78631088db0000647f5c764cbc14f9669b88837ca1490cca17c316070000647f5c764cbc14f9669b88837ca1490cca17c31607000000000000000000000000000000000000000000000000000000000000', '0x000000000000000000000000533cf9fb379488ffe0b1065c42c744fbd4b0e1a30000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000c00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000299560e827af36c94d2ac33a39bce1fe78631088db007f5c764cbc14f9669b88837ca1490cca17c316070000000000000000000000000000000000000000000000'])

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()